In [92]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
pd.options.display.max_rows = None
train_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [93]:
depVar = train_df['SalePrice']
train_df.drop(columns=['SalePrice', 'Id', 'Utilities'], inplace=True)
test_df.drop(columns=['Utilities'], inplace=True)

In [94]:
train_df.dtypes

MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
Alley             object
LotShape          object
LandContour       object
LotConfig         object
LandSlope         object
Neighborhood      object
Condition1        object
Condition2        object
BldgType          object
HouseStyle        object
OverallQual        int64
OverallCond        int64
YearBuilt          int64
YearRemodAdd       int64
RoofStyle         object
RoofMatl          object
Exterior1st       object
Exterior2nd       object
MasVnrType        object
MasVnrArea       float64
ExterQual         object
ExterCond         object
Foundation        object
BsmtQual          object
BsmtCond          object
BsmtExposure      object
BsmtFinType1      object
BsmtFinSF1         int64
BsmtFinType2      object
BsmtFinSF2         int64
BsmtUnfSF          int64
TotalBsmtSF        int64
Heating           object
HeatingQC         object
CentralAir        object


In [95]:
# Check for NaN
train_df.isnull().sum().sum(), test_df.isnull().sum().sum() 

(7829, 7876)

In [96]:
continuous = []
for col in train_df.columns:
    if train_df[col].dtype != object:
        continuous.append(col)

In [97]:
categorical = []
for col in train_df.columns:
    if train_df[col].dtype == object:
        categorical.append(col)

In [98]:
len(continuous), len(categorical)

(36, 42)

In [99]:
# Fill missing with median in the numerical columns
for col in continuous:
    train_df[col] = train_df[col].fillna(train_df[col].median())
    test_df[col] = test_df[col].fillna(test_df[col].median())
# Fill missing with mode in the numerical columns
for col in categorical:
    train_mode = train_df[col].mode()[0]
    train_df[col] = train_df[col].fillna(train_mode)
    test_mode = test_df[col].mode()[0]
    test_df[col] = test_df[col].fillna(test_mode)

In [100]:
# Check for NaN
train_df.isnull().sum().sum(), test_df.isnull().sum().sum() 

(0, 0)

In [101]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_df[continuous] = scaler.fit_transform(train_df[continuous])
test_df[continuous] = scaler.fit_transform(test_df[continuous])

In [102]:
train_df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,0.073375,RL,-0.220875,-0.207142,Pave,Grvl,Reg,Lvl,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,0.651479,-0.517200,1.050994,0.878668,Gable,CompShg,VinylSd,VinylSd,BrkFace,0.514104,Gd,TA,PConc,Gd,TA,No,GLQ,0.575425,Unf,-0.288653,-0.944591,-0.459303,GasA,Ex,Y,SBrkr,-0.793434,1.161852,-0.120242,0.370333,1.107810,-0.241061,0.789741,1.227585,0.163779,-0.211454,Gd,0.912210,Typ,-0.951226,Gd,Attchd,1.017598,RFn,0.311725,0.351000,TA,TA,Y,-0.752176,0.216503,-0.359325,-0.116339,-0.270208,-0.068692,Gd,MnPrv,Shed,-0.087688,-1.599111,0.138777,WD,Normal
1,-0.872563,RL,0.460320,-0.091886,Pave,Grvl,Reg,Lvl,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,-0.071836,2.179628,0.156734,-0.429577,Gable,CompShg,MetalSd,MetalSd,BrkFace,-0.570750,TA,TA,CBlock,Gd,TA,Gd,ALQ,1.171992,Unf,-0.288653,-0.641228,0.466465,GasA,Ex,Y,SBrkr,0.257140,-0.795163,-0.120242,-0.482512,-0.819964,3.948809,0.789741,-0.761621,0.163779,-0.211454,TA,-0.318683,Typ,0.600495,TA,Attchd,-0.107927,RFn,0.311725,-0.060731,TA,TA,Y,1.626195,-0.704483,-0.359325,-0.116339,-0.270208,-0.068692,Gd,MnPrv,Shed,-0.087688,-0.489110,-0.614439,WD,Normal
2,0.073375,RL,-0.084636,0.073480,Pave,Grvl,IR1,Lvl,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,0.651479,-0.517200,0.984752,0.830215,Gable,CompShg,VinylSd,VinylSd,BrkFace,0.325915,Gd,TA,PConc,Gd,TA,Mn,GLQ,0.092907,Unf,-0.288653,-0.301643,-0.313369,GasA,Ex,Y,SBrkr,-0.627826,1.189351,-0.120242,0.515013,1.107810,-0.241061,0.789741,1.227585,0.163779,-0.211454,Gd,-0.318683,Typ,0.600495,TA,Attchd,0.934226,RFn,0.311725,0.631726,TA,TA,Y,-0.752176,-0.070361,-0.359325,-0.116339,-0.270208,-0.068692,Gd,MnPrv,Shed,-0.087688,0.990891,0.138777,WD,Normal
3,0.309859,RL,-0.447940,-0.096897,Pave,Grvl,IR1,Lvl,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,0.651479,-0.517200,-1.863632,-0.720298,Gable,CompShg,Wd Sdng,Wd Shng,BrkFace,-0.570750,TA,TA,BrkTil,TA,Gd,No,ALQ,-0.499274,Unf,-0.288653,-0.061670,-0.687324,GasA,Gd,Y,SBrkr,-0.521734,0.937276,-0.120242,0.383659,1.107810,-0.241061,-1.026041,-0.761621,0.163779,-0.211454,Gd,0.296763,Typ,0.600495,Gd,Detchd,0.809167,Unf,1.650307,0.790804,TA,TA,Y,-0.752176,-0.176048,4.092524,-0.116339,-0.270208,-0.068692,Gd,MnPrv,Shed,-0.087688,-1.599111,-1.367655,WD,Abnorml
4,0.073375,RL,0.641972,0.375148,Pave,Grvl,IR1,Lvl,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,1.374795,-0.517200,0.951632,0.733308,Gable,CompShg,VinylSd,VinylSd,BrkFace,1.366489,Gd,TA,PConc,Gd,TA,Av,GLQ,0.463568,Unf,-0.288653,-0.174865,0.199680,GasA,Ex,Y,SBrkr,-0.045611,1.617877,-0.120242,1.299326,1.107810,-0.241061,0.789741,1.227585,1.390023,-0.211454,Gd,1.527656,Typ,0.600495,TA,Attchd,0.892540,RFn,1.650307,1.698485,TA,TA,Y,0.780197,0.563760,-0.359325,-0.116339,-0.270208,-0.068692,Gd,MnPrv,Shed,-0.087688,2.100892,0.138777,WD,Normal


In [103]:
from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()

for col in categorical:
    train_df[col] = train_df[col].astype(str)
    train_df[col] = LE.fit_transform(train_df[col])
    test_df[col] = test_df[col].astype(str)
    test_df[col] = LE.fit_transform(test_df[col])

In [104]:
columns = continuous + categorical
X = train_df[columns]
y = depVar
X.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,0.073375,-0.220875,-0.207142,0.651479,-0.517200,1.050994,0.878668,0.514104,0.575425,-0.288653,-0.944591,-0.459303,-0.793434,1.161852,-0.120242,0.370333,1.107810,-0.241061,0.789741,1.227585,0.163779,-0.211454,0.912210,-0.951226,1.017598,0.311725,0.351000,-0.752176,0.216503,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-1.599111,0.138777,3,1,0,3,3,4,0,5,2,2,0,5,1,1,12,13,1,2,4,2,2,3,3,2,5,1,0,1,4,2,6,2,1,1,4,4,2,2,2,2,8,4
1,-0.872563,0.460320,-0.091886,-0.071836,2.179628,0.156734,-0.429577,-0.570750,1.171992,-0.288653,-0.641228,0.466465,0.257140,-0.795163,-0.120242,-0.482512,-0.819964,3.948809,0.789741,-0.761621,0.163779,-0.211454,-0.318683,0.600495,-0.107927,0.311725,-0.060731,1.626195,-0.704483,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-0.489110,-0.614439,3,1,0,3,3,2,0,24,1,2,0,2,1,1,8,8,1,3,4,1,2,3,1,0,5,1,0,1,4,3,6,4,1,1,4,4,2,2,2,2,8,4
2,0.073375,-0.084636,0.073480,0.651479,-0.517200,0.984752,0.830215,0.325915,0.092907,-0.288653,-0.301643,-0.313369,-0.627826,1.189351,-0.120242,0.515013,1.107810,-0.241061,0.789741,1.227585,0.163779,-0.211454,-0.318683,0.600495,0.934226,0.311725,0.631726,-0.752176,-0.070361,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,0.990891,0.138777,3,1,0,0,3,4,0,5,2,2,0,5,1,1,12,13,1,2,4,2,2,3,2,2,5,1,0,1,4,2,6,4,1,1,4,4,2,2,2,2,8,4
3,0.309859,-0.447940,-0.096897,0.651479,-0.517200,-1.863632,-0.720298,-0.570750,-0.499274,-0.288653,-0.061670,-0.687324,-0.521734,0.937276,-0.120242,0.383659,1.107810,-0.241061,-1.026041,-0.761621,0.163779,-0.211454,0.296763,0.600495,0.809167,1.650307,0.790804,-0.752176,-0.176048,4.092524,-0.116339,-0.270208,-0.068692,-0.087688,-1.599111,-1.367655,3,1,0,0,3,0,0,6,2,2,0,5,1,1,13,15,1,3,4,0,3,1,3,0,5,1,2,1,4,2,6,2,5,2,4,4,2,2,2,2,8,0
4,0.073375,0.641972,0.375148,1.374795,-0.517200,0.951632,0.733308,1.366489,0.463568,-0.288653,-0.174865,0.199680,-0.045611,1.617877,-0.120242,1.299326,1.107810,-0.241061,0.789741,1.227585,1.390023,-0.211454,1.527656,0.600495,0.892540,1.650307,1.698485,0.780197,0.563760,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,2.100892,0.138777,3,1,0,0,3,2,0,15,2,2,0,5,1,1,12,13,1,2,4,2,2,3,0,2,5,1,0,1,4,2,6,4,1,1,4,4,2,2,2,2,8,4


In [105]:
X.shape, y.shape

((1460, 78), (1460,))

In [106]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [107]:
# Hyper Parameter Optimization
params = {
    "learning_rate": [0.01, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30],  # Step size shrinkage to prevent overfitting
    "max_depth": [3, 4, 5, 6, 8, 10, 12, 15],  # Maximum depth of a tree
    "min_child_weight": [1, 3, 5, 7],  # Minimum sum of instance weight (hessian) needed in a child
    "gamma": [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],  # Minimum loss reduction required to make a further partition
    "colsample_bytree": [0.3, 0.4, 0.5, 0.7, 0.8],  # Subsample ratio of columns when constructing each tree
    "subsample": [0.6, 0.7, 0.8, 0.9, 1.0],  # Subsample ratio of the training instances
    "n_estimators": [100, 200, 300, 400, 500],  # Number of boosting rounds
    "reg_alpha": [0, 0.01, 0.05, 0.1, 1, 10],  # L1 regularization term on weights
    "reg_lambda": [0.01, 0.05, 0.1, 1, 10],  # L2 regularization term on weights
    "scale_pos_weight": [1, 1.5, 2, 2.5, 3],  # Controls the balance of positive and negative weights
    "colsample_bylevel": [0.6, 0.7, 0.8, 0.9, 1.0],  # Subsample ratio of columns for each split
}

In [108]:
import sklearn
sklearn.metrics.get_scorer_names()

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_negative_likelihood_ratio',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'positive_likelihood_ratio',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',

In [109]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost

classifier = xgboost.XGBRegressor()
random_search = RandomizedSearchCV(classifier, param_distributions=params, n_iter=5, scoring= 'neg_mean_squared_error', n_jobs=-1, cv=5, verbose=3)

In [110]:
random_search.fit(X,y)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
                                        'gamma': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
                                        'learning_rate': [0.01, 0.05, 0.1, 0.15,
                                                          0.2, 0.25, 0.3],
                                        'max_depth': [3, 4, 5, 6, 8, 10, 12,
                                                      15],
                                        'min_child_weight': [1, 3, 5, 7],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500],
                                        'reg_alpha': [0, 0.01, 0.05, 0.1, 1,
                                                      10],
                                        'reg_lambda': [0.01, 0.05, 0.1, 1, 10],
                                        'scale_pos_weight': [1, 1.5, 2, 2.5, 3],
                                        'subsample': [0.6, 0.7, 0.8, 0.9, 1.0]},
                   scoring='neg_mean_squared_error', verbose=3)

In [111]:
random_search.best_estimator_

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=0.6, colsample_bynode=None, colsample_bytree=0.7,
             device=None, early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0.1, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.01, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=12,
             max_leaves=None, min_child_weight=3, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=500,
             n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [112]:
random_search.best_params_

{'subsample': 0.7,
 'scale_pos_weight': 1,
 'reg_lambda': 0.05,
 'reg_alpha': 1,
 'n_estimators': 500,
 'min_child_weight': 3,
 'max_depth': 12,
 'learning_rate': 0.01,
 'gamma': 0.1,
 'colsample_bytree': 0.7,
 'colsample_bylevel': 0.6}

In [113]:
from xgboost import XGBRegressor

model = XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=0.9, colsample_bynode=None, colsample_bytree=0.7,
             device=None, early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0.4, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
             max_leaves=None, min_child_weight=3, missing=np.nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=400,
             n_jobs=None, num_parallel_tree=None, random_state=None)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=0.9, colsample_bynode=None, colsample_bytree=0.7,
             device=None, early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0.4, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
             max_leaves=None, min_child_weight=3, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=400,
             n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [114]:
from sklearn.metrics import mean_squared_error

valid_predictions = model.predict(X_valid)
# y_valid = y_valid.values
rmse = np.sqrt(mean_squared_error(y_valid, valid_predictions))
rmse

24998.20263677757

In [115]:
# Reordering the columns to be able to predict
cols_when_model_builds = model.get_booster().feature_names
test_df = test_df[cols_when_model_builds]

In [116]:
test_df.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,-0.874711,0.567330,0.363929,-0.751101,0.400766,-0.340945,-1.072885,-0.563316,0.063428,0.517537,-0.650461,-0.370716,-0.654561,-0.775254,-0.080483,-1.215588,-0.818068,-0.25816,-1.028720,-0.751040,-1.029543,-0.20391,-0.918335,-0.898055,-0.653107,-0.988189,1.185921,0.366678,-0.701628,-0.360738,-0.088827,1.818960,-0.057227,-0.092244,-0.038281,1.713905,2,1,0,3,3,4,0,12,1,2,0,2,1,0,10,12,1,3,4,1,3,3,3,4,3,0,4,1,3,3,6,2,1,2,3,4,2,0,2,2,8,4
1,-0.874711,0.615963,0.897861,-0.054877,0.400766,-0.439695,-1.214908,0.047057,1.063511,-0.297689,-0.339225,0.639230,0.433298,-0.775254,-0.080483,-0.323539,-0.818068,-0.25816,-1.028720,1.237648,0.175997,-0.20391,-0.255371,-0.898055,-0.769806,-0.988189,-0.741235,2.347867,-0.178826,-0.360738,-0.088827,-0.301543,-0.057227,19.730438,-0.038281,1.713905,3,1,0,0,3,0,0,12,2,2,0,2,3,0,11,13,1,3,4,1,3,3,3,0,5,0,4,1,3,2,6,2,1,2,3,4,2,0,2,0,8,4
2,0.061351,0.275532,0.809646,-0.751101,-0.497418,0.844059,0.678742,-0.563316,0.773377,-0.297689,-0.954831,-0.266784,-0.574165,0.891944,-0.080483,0.294508,-0.818068,-0.25816,0.773083,1.237648,0.175997,-0.20391,-0.255371,0.647066,0.747276,0.301406,0.042537,0.930495,-0.207871,-0.360738,-0.088827,-0.301543,-0.057227,-0.092244,-1.140614,1.713905,3,1,0,0,3,4,0,8,2,2,0,4,1,0,10,12,1,3,4,2,2,3,3,2,5,0,2,1,3,3,6,4,1,0,3,4,2,0,2,2,8,4
3,0.061351,0.470064,0.032064,-0.054877,0.400766,0.876976,0.678742,-0.450284,0.357958,-0.297689,-0.526882,-0.271303,-0.579190,0.837243,-0.080483,0.243004,-0.818068,-0.25816,0.773083,1.237648,0.175997,-0.20391,0.407593,0.647066,0.786176,0.301406,-0.012788,2.089451,-0.178826,-0.360738,-0.088827,-0.301543,-0.057227,-0.092244,-0.038281,1.713905,3,1,0,0,3,4,0,8,2,2,0,4,1,0,10,12,1,3,4,2,3,3,3,2,5,0,0,1,3,2,6,2,1,0,3,4,2,0,2,2,8,4
4,1.465443,-1.232092,-0.971808,1.337571,-0.497418,0.679475,0.394694,-0.563316,-0.387160,-0.297689,1.059048,0.528520,0.310192,-0.775254,-0.080483,-0.424487,-0.818068,-0.25816,0.773083,-0.751040,-1.029543,-0.20391,-0.918335,-0.898055,0.552779,0.301406,0.153187,-0.729632,0.489198,-0.360738,-0.088827,2.243060,-0.057227,-0.092244,-1.875504,1.713905,3,1,0,0,1,4,0,22,2,2,4,2,1,0,6,6,1,2,4,2,2,3,3,0,5,0,0,1,3,2,6,2,1,1,3,4,2,0,2,2,8,4


In [117]:
preds = model.predict(test_df)
preds

array([129004.51, 158293.44, 175283.95, ..., 162824.08, 121534.81,
       212493.2 ], dtype=float32)

In [118]:
test_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
submission_csv = pd.DataFrame({
    'Id': test_df['Id'],
    'SalePrice': preds
})
submission_csv.head()

,Id,SalePrice
0,1461,129004.507812
1,1462,158293.437500
2,1463,175283.953125
3,1464,178351.843750
4,1465,184182.562500


In [119]:
submission_csv.to_csv('submission.csv', index=False)